In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the Data Paths
TRAIN_PATH = '/content/drive/MyDrive/images1/train'
VAL_PATH = '/content/drive/MyDrive/images1/val'
TEST_PATH="/content/drive/MyDrive/images1/test"


In [4]:
# Define parameters
img_size = (299, 299, 3)
num_classes = 2  # Binary classification: Glaucoma Negative and Glaucoma Positive
epochs = 60
batch_size = 32

# Custom Data Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

class CustomImageDataGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def flow_from_directory(self, directory, *args, **kwargs):
        try:
            return super().flow_from_directory(directory, *args, **kwargs)
        except Exception as e:
            print(f"Error processing images in directory: {directory}. Skipping...")
            print(e)
            return None  # Return None if there's an error


In [5]:
from PIL import Image
import os

def validate_images(directory):
    invalid_images = []

    # Iterate through all files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            try:
                # Open the image file
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img.close()  # Close the image file

            except Exception as e:
                # Add the path of the invalid image to the list
                invalid_images.append(img_path)
                print(f"Error processing image: {img_path}. Skipping...")
                print(e)

    return invalid_images

# Directory containing the images
directory = '/content/drive/MyDrive/images1/train'

# Validate images
invalid_images = validate_images(directory)

# Print list of invalid images
print("Invalid images:")
for img_path in invalid_images:
    print(img_path)


Invalid images:


In [6]:
# Data generators
train_datagen = CustomImageDataGenerator(
    rescale=1./255,
    rotation_range=60,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = CustomImageDataGenerator(rescale=1./255)
test_datagen = CustomImageDataGenerator(rescale=1./255)

In [7]:
# Flow from directory
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode='binary'  # Binary classification
)

val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode='binary', # Binary classification
    color_mode='rgb',
    seed=2019
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    color_mode='rgb',
    shuffle=False,  # Set to False for test set
    seed=2019
)

Found 920 images belonging to 2 classes.
Found 114 images belonging to 2 classes.
Found 117 images belonging to 2 classes.


In [8]:
# Model
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=img_size)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=0.000001)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


87910968/87910968 [==============================] - 1s 0us/step


In [9]:
# Ensure train_generator is defined and initialized
if train_generator is None or not train_generator:
    raise ValueError("`train_generator` is not properly initialized. Ensure it is defined and contains data.")

# Model training
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks=[reduce_lr, early_stop]
)

Epoch 1/60
29/29 [==============================] - 136s 3s/step - loss: 0.9821 - accuracy: 0.5163 - val_loss: 940.8917 - val_accuracy: 0.4912 - lr: 0.0010
Epoch 2/60
29/29 [==============================] - 47s 2s/step - loss: 0.7280 - accuracy: 0.5326 - val_loss: 0.7456 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/60
29/29 [==============================] - 48s 2s/step - loss: 0.7091 - accuracy: 0.5326 - val_loss: 44.3975 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/60
29/29 [==============================] - 46s 2s/step - loss: 0.7080 - accuracy: 0.5120 - val_loss: 14.1125 - val_accuracy: 0.4912 - lr: 0.0010
Epoch 5/60
29/29 [==============================] - ETA: 0s - loss: 0.6714 - accuracy: 0.6000
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
29/29 [==============================] - 48s 2s/step - loss: 0.6714 - accuracy: 0.6000 - val_loss: 1.0224 - val_accuracy: 0.6579 - lr: 0.0010
Epoch 6/60
29/29 [==============================] - 50s 2s/step - loss

In [10]:
 # Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator, verbose=1)

# Print the validation accuracy
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))


4/4 [==============================] - 2s 528ms/step - loss: 0.2971 - accuracy: 0.8596
Validation Accuracy: 85.96%


In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image you want to predict
image_path = '/content/drive/MyDrive/images1/test/glaucoma/Im332_g_ACRIMA.jpg'  
img = image.load_img(image_path, target_size=(299, 299))  

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Rescale to [0, 1] as done in training

# Perform prediction
prediction = model.predict(img_array)

# Map prediction to classes
if prediction < 0.5:
    print("Prediction: Glaucoma Positive")
else:
    print("Prediction: Glaucoma Negative")


1/1 [==============================] - 3s 3s/step
Prediction: Glaucoma Positive
